In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import rasterio
import rioxarray as rioxr
import geopandas as gpd

import planetary_computer as pc

from shapely.geometry import Polygon

import data_sampling_workflow.utility as utility

In [2]:
itemid = 'ca_m_3411934_sw_11_060_20200521'
# Fields next to Goleta Slough
reduce_box = Polygon([[-119.8284196946,34.4162731913],
                       [-119.8101541026,34.4162731913],
                       [-119.8101541026,34.4353838099],
                       [-119.8284196946,34.4353838099],
                       [-119.8284196946,34.4162731913]])

reduce_box_crs="EPSG:4326"

In [3]:
item = utility.get_item_from_id(itemid)
item.datetime

datetime.datetime(2020, 5, 21, 0, 0, tzinfo=tzlocal())

In [ ]:
type(item.datetime)

In [ ]:
type(item.datetime.date())

In [4]:
# ***************************************************************************************************
# ***************************************************************************************************

def rioxr_from_itemid(itemid, reduce_box = False, reduce_box_crs = False):
    item = utility.get_item_from_id(itemid)
    href = pc.sign(item.assets["image"].href)
    
    rast = rioxr.open_rasterio(href)
    
    if reduce_box != False:
        reduce = gpd.GeoDataFrame({'geometry':[reduce_box]}, crs=reduce_box_crs)
        reduce = reduce.to_crs(rast.rio.crs)        
        rast = rast.rio.clip_box(*reduce.total_bounds)
    
    rast.attrs['datetime'] = item.datetime
    
    return rast

In [5]:
rast = rioxr_from_itemid(itemid)
rast

<xarray.DataArray (band: 4, y: 12500, x: 10580)>
[529000000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 2.354e+05 2.354e+05 ... 2.418e+05 2.418e+05
  * y            (y) float64 3.815e+06 3.815e+06 ... 3.807e+06 3.807e+06
    spatial_ref  int64 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0
    datetime:      2020-05-21 00:00:00+00:00

In [ ]:
rast.attrs['datetime'] == item.datetime

In [ ]:
type(rast)

In [ ]:
rast.rio.crs

In [ ]:
rast.rio.transform()

In [ ]:
rast_small = rioxr_from_itemid(itemid, reduce_box, reduce_box_crs)
rast_small

In [ ]:
rast_small.rio.crs

In [ ]:
rast_small.rio.transform()

In [6]:
# ***************************************************************************************************
# ***************************************************************************************************

def raster_as_df(raster, band_names):
    """
             Parameters:
       
            Returns: 
    """ 
    
    pixels = raster.reshape([len(band_names),-1]).T
    df = pd.DataFrame(pixels, columns=band_names) 
    return df

In [7]:
pixels = raster_as_df(rast.to_numpy(),  ['r','g','b','nir'])

In [8]:
def normalized_difference_index(df, *args):
    m = args[0]
    n = args[1]
    
    x = df.iloc[:, m].astype('int16')  
    y = df.iloc[:, n].astype('int16')
    return (x-y) / (x+y)

In [ ]:
ndvi = normalized_difference_index(pixels,3,0)
ndvi

In [9]:
def feature_df_treshold(df, feature_name, thresh, keep_gr, func, *args):
    
    df[feature_name] = func(df, *args)
    
    if keep_gr == True:
        keep = df[df[feature_name] > thresh]
        deleted_indices = df[df[feature_name] <= thresh].index
    else : 
        keep = df[df[feature_name] < thresh]
        deleted_indices = df[df[feature_name] >= thresh].index
        
    deleted_indices = deleted_indices.to_numpy()
    
    return keep, deleted_indices

In [10]:
not_water, water_index = feature_df_treshold(pixels, 'ndwi', 0.3, False, normalized_difference_index, 1,3)

In [11]:
is_veg, non_veg_index = feature_df_treshold(not_water, 'ndvi', 0.05, True, normalized_difference_index, 3,0)

/tmp/ipykernel_3167/312715764.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature_name] = func(df, *args)


In [ ]:
is_veg

In [12]:
water_index

array([       13,        75,        76, ..., 132249997, 132249998,
       132249999])

In [13]:
non_veg_index

array([        0,         1,         2, ..., 132220804, 132228346,
       132231481])

In [14]:
def indices_to_image(nrows, ncols, indices_list, values, back_value):
    # background, any pixel not in the union of indices will be given this value
    reconstruct = np.ones((nrows,ncols))*back_value 

    # TO DO: check indices list and values lengths are the same?
    for k in range(0,len(indices_list)):
        i = indices_list[k] / ncols
        i = i.astype(int)
        j = indices_list[k] % ncols
        reconstruct[i,j] = values[k]
    
    return reconstruct

In [17]:
reconstruct = indices_to_image(12500, 10580, [water_index, non_veg_index], [3,2], back_value=1)

In [19]:
utility.save_raster(reconstruct, 
                    os.getcwd()+'/trial.tif', 
                    (rast.shape[1],rast.shape[2]), 
                    1, 
                    rast.rio.crs, 
                    rast.rio.transform(), 'int16' )
